<a href="https://colab.research.google.com/github/nikolmash/yes-no-question-answering/blob/master/QA_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install jsonlines
import numpy as np
import pandas as pd
import tensorflow as tf 
import torch
import jsonlines
from transformers import *
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

     |████████████████████████████████| 645kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 14.0MB/s 
     |████████████████████████████████| 890kB 40.2MB/s 
     |████████████████████████████████| 1.1MB 41.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5374bca39cc863460b1e868d8ac9a224e7202de2abb4dcf1dc77e25094570910
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Если я загружаю корпус по ссылке из гитхаба, то у меня почему-то постоянно падает открытие этого файла .jsonl , поэтому открываю со своего гугл-диска

In [0]:
with jsonlines.open('/content/drive/My Drive/train.jsonl') as reader:
      train = [obj for obj in reader]
with jsonlines.open('/content/drive/My Drive/dev.jsonl') as reader:
      dev = [obj for obj in reader]

In [4]:
train_df = pd.DataFrame(train)
train_df.head()

,question,title,answer,passage
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,Good Samaritan law,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,Windows Movie Maker,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,Powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,False,As with other games in The Elder Scrolls serie...


In [5]:
dev_df = pd.DataFrame(dev)
dev_df.head()

,question,title,answer,passage
0,does ethanol take more energy make that produces,Ethanol fuel,False,All biomass goes through at least some of thes...
1,is house tax and property tax are same,Property tax,True,Property tax or 'house tax' is a local tax on ...
2,is pain experienced in a missing body part or ...,Phantom pain,True,Phantom pain sensations are described as perce...
3,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts,True,Harry Potter and the Escape from Gringotts is ...
4,is there a difference between hydroxyzine hcl ...,Hydroxyzine,True,Hydroxyzine preparations require a doctor's pr...


In [6]:
bert_model = TFBertModel.from_pretrained("bert-base-uncased") 
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [0]:
train_df['data'] = "[CLS] " + train_df['question'] + " [SEP] " + train_df['passage'] + " [SEP]" 
dev_df['data'] = "[CLS] " + dev_df['question'] + " [SEP] " + dev_df['passage'] + " [SEP]" 

In [0]:
train_df['data'] = train_df['data'].apply(bert_tokenizer.tokenize)
dev_df['data'] = dev_df['data'].apply(bert_tokenizer.tokenize)

In [0]:
max_length = 512
train_input_ids = []
train_attention_masks = []

for text in train_df['data'].values:
  encoded_data = bert_tokenizer.encode_plus(text, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
  train_input_ids.append(encoded_data["input_ids"])
  train_attention_masks.append(encoded_data["attention_mask"])

train_input_ids = np.array(train_input_ids)
train_attention_masks = np.array(train_attention_masks)

In [0]:
dev_input_ids = []
dev_attention_masks = []

for text in dev_df['data'].values:
  encoded_data = bert_tokenizer.encode_plus(text, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
  dev_input_ids.append(encoded_data["input_ids"])
  dev_attention_masks.append(encoded_data["attention_mask"])

dev_input_ids = np.array(dev_input_ids)
dev_attention_masks = np.array(dev_attention_masks)

In [0]:
train_answers = train_df['answer'].values.astype(int)
dev_answers = dev_df['answer'].values.astype(int)

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_input_ids, train_attention_masks, train_answers))
dev_dataset = tf.data.Dataset.from_tensor_slices((dev_input_ids, dev_attention_masks, dev_answers))

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [0]:
history = bert_model.fit(train_dataset, epochs=2, steps_per_epoch=100,
                    validation_data=dev_dataset, validation_steps=20)

Epoch 1/2
100/100 [==============================] - 302s 3s/step - loss: 4.6359 - output_1_loss: 0.8433 - output_2_loss: 3.7926 - output_1_accuracy: 0.8149 - output_2_accuracy: 0.2340 - val_loss: 4.6485 - val_output_1_loss: 0.0028 - val_output_2_loss: 4.6457 - val_output_1_accuracy: 0.9998 - val_output_2_accuracy: 0.8945
Epoch 2/2
100/100 [==============================] - 302s 3s/step - loss: 3.1519 - output_1_loss: 0.0491 - output_2_loss: 3.1029 - output_1_accuracy: 0.9978 - output_2_accuracy: 0.9089 - val_loss: 4.3642 - val_output_1_loss: 0.0056 - val_output_2_loss: 4.3586 - val_output_1_accuracy: 0.9998 - val_output_2_accuracy: 0.9997


Тут происходит что-то очень странное. В туториале от huggingface предлагают сохранить обученную модель на тензорфлоу, а потом открыть ее с помощью пайторча (?). https://github.com/huggingface/transformers#Quick-tour-TF-20-training-and-PyTorch-interoperability  Тензорфлоу в режиме предсказания постоянно падает(

In [0]:
bert_model.save_pretrained('/content/drive/My Drive/')

In [0]:
pytorch_model = BertModel.from_pretrained('/content/drive/My Drive/', from_tf=True)

Достаю из модели эмбеддинги для обучающей и валидационной выборки. Отдаю их в модель "пачками" по 100 элементов. Если отдавать целиком - то падает, а по одному ячейки работают невероятно долго, хотя и сейчас тоже около трех часов

In [13]:
train_embeddings = []
for i in tqdm(range(100, len(train_input_ids), 100)):
  train_ids = torch.tensor(train_input_ids[i-100:i])
  with torch.no_grad():
    last_hidden_states = pytorch_model(train_ids)[0]
  train_embeddings.append(last_hidden_states.numpy()[:,:,-1])
train_ids = torch.tensor(train_input_ids[i:])
with torch.no_grad():
  last_hidden_states = pytorch_model(train_ids)[0]
train_embeddings.append(last_hidden_states.numpy()[:,:,-1])

In [14]:
dev_embeddings = []
for i in tqdm(range(100, len(dev_input_ids), 100)):
  test_ids = torch.tensor(dev_input_ids[i-100:i])
  with torch.no_grad():
    last_hidden_states = pytorch_model(test_ids)[0]
  dev_embeddings.append(last_hidden_states.numpy()[:,:,-1])
train_ids = torch.tensor(dev_input_ids[i:])
with torch.no_grad():
  last_hidden_states = pytorch_model(train_ids)[0]
dev_embeddings.append(last_hidden_states.numpy()[:,:,-1])


Конкатенируем "пачки" эмбеддингов по 100 в один нормальный numpy array

In [39]:
train_data = np.concatenate(train_embeddings)
dev_data = np.concatenate(dev_embeddings)
print(train_data.shape)
print(dev_data.shape)

(9427, 512)
(3270, 512)


Обучаем логистическую регрессию

In [0]:
lr = LogisticRegression(max_iter=1000)
lr.fit(train_data, train_answers)
predictions = lr.predict(dev_data)

In [42]:
print('Accuracy: ', accuracy_score(dev_answers, predictions))
print('F1 score: ', f1_score(dev_answers, predictions))

Accuracy:  0.6122324159021407
F1 score:  0.7444578798871423
